In [1]:
#Importing the necessary libraries for the script
import geemap
import ee
from pathlib import Path
import geopandas as gpd
from functions import *

In [2]:
#Authenticating the Google Earth Engine API
ee.Authenticate()
ee.Initialize()
geemap.ee_initialize()

In [3]:
#Setting the Out Directory if not exists creating it
out_dir = Path('../Data')
out_dir.mkdir(exist_ok=True)

#Shapefile directory of the area of interest
shp_dir = Path('../Shapefile/BegnasArea.shp')

In [6]:
studyarea = get_eesupported_roi(shp_dir)

#Earth Engine Collection
dataset_landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA').filterDate('2022-04-01', '2022-07-01').filter(ee.Filter.lt('CLOUD_COVER', 10))
dataset_landsat9 = ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterDate('2022-04-01', '2022-07-01').filter(ee.Filter.lt('CLOUD_COVER', 10))

In [19]:
#Downloading the image of Landsat 8
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )
  
dataset_landsat9 = dataset_landsat9.map(apply_scale_factors)
landsat_9 = dataset_landsat9.median().clip(studyarea)
save_data(out_dir, landsat_9, 'landsat_91.tif', studyarea)

#Downloading the image of Landsat 9
landsat_9 = dataset_landsat9.median().clip(studyarea)
save_data(out_dir, landsat_9, 'landsat_9.tif', studyarea)

landsat_8.tif: |          | 0.00/4.43M (raw) [  0.0%] in 00:00 (eta:     ?)

landsat_9.tif: |          | 0.00/4.95M (raw) [  0.0%] in 00:00 (eta:     ?)